In [ ]:
%load_ext autoreload
%autoreload 2



import geopandas as gpd
import folium
import folium.plugins
import logging
from eo_tools.util import explore_products
from eo_tools.S2 import process_s2_tiles
import rioxarray as riox


from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)



# Directory setup

In [1]:
# change to your data directories
path_data = "/data"
path_s2 = f"{path_data}/S2"
path_processed = f"{path_data}/res"

## Select an area of interest

- Select a drawing tool (polygon, circle or rectangle)
- Draw the AOI on the map
- Click "Export" on the right to save the GeoJSON file

In [2]:
map_select = folium.Map()
folium.plugins.Draw(
    export=True,
    draw_options={
        "polyline": False,
        "circlemarker": False,
        "marker": False,
    },
).add_to(map_select)
map_select

NameError: name 'folium' is not defined

# Search

In [ ]:
# load a geometry
file_aoi = "../data/Bretagne_AOI.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
search_criteria = {
    "productType": "S2_MSI_L1C",
    "start": "2023-06-01",
    "end": "2023-08-31",
    "cloudCover": 10,
    "geom": shp
}
results = dag.search_all(**search_criteria)

Getting unique dates to help with product exploration

In [ ]:
def get_unique_dates(results):
    import numpy as np

    dates = []
    for p in results:
        dates.append(p.properties["startTimeFromAscendingNode"])
    return np.unique(dates).tolist()

dates = get_unique_dates(results)
print(f"unique dates:")
print(", \n".join(dates))

# Visualize product footprints on a map

In [ ]:
# show all results
# explore_products(results, shp)

# filter by date
results_filtered = results.filter_date("2023-06-05", "2023-06-06") 
explore_products(results_filtered, shp)

# Download

In [ ]:
# re-run to get only links
dl = dag.download_all(results_filtered, outputs_prefix=path_s2)

# Merge downloaded tiles

In [ ]:
out_dirs = process_s2_tiles(
    dl,
    bands=["B2", "B3", "B4", "B8"],
    shp=shp,
    aoi_name="aoi_bretagne",
    outputs_prefix=path_processed,
)

# Visualize

In [ ]:
pth_arr = f"{out_dirs[0]}/B8.tif"
arr = riox.open_rasterio(pth_arr)
arr[0].plot.imshow(cmap="gray", vmax=1)